# Morphological analysis with premorph and postmorph

In [1]:
from estnltk.text import Text

By default premorph and postmorph are executed.

In [2]:
t = Text('tu-li 3ndaks').tag_layer()
t.morf_analysis

SL[SL[Span(tu-li, {'clitic': '', 'root_tokens': ['tule'], 'root': 'tule', 'ending': 'i', 'partofspeech': 'V', 'form': 's', 'lemma': 'tulema'})],
SL[Span(3ndaks, {'clitic': '', 'root_tokens': ['3'], 'root': '3', 'ending': 'ks', 'partofspeech': 'O', 'form': 'sg tr', 'lemma': '3'})]]

This is equivalent to

In [3]:
from estnltk.taggers.premorph.premorf import WordNormalizingTagger
from estnltk.taggers.morf import VabamorfTagger
from estnltk.rewriting.postmorph.vabamorf_corrector import VabamorfCorrectionRewriter

vabamorf_corrector = VabamorfCorrectionRewriter(replace=True)

t = Text('tu-li 3ndaks')
t.tag_layer(['words'])
WordNormalizingTagger().tag(t)
VabamorfTagger(premorf_layer='normalized', postmorph_rewriter=vabamorf_corrector).tag(t)

t.morf_analysis

SL[SL[Span(tu-li, {'clitic': '', 'root_tokens': ['tule'], 'root': 'tule', 'ending': 'i', 'partofspeech': 'V', 'form': 's', 'lemma': 'tulema'})],
SL[Span(3ndaks, {'clitic': '', 'root_tokens': ['3'], 'root': '3', 'ending': 'ks', 'partofspeech': 'O', 'form': 'sg tr', 'lemma': '3'})]]

Modify the code above to customize premorph or postmorph.

For example, turn off premorph and postmorph:

In [4]:
t = Text('tu-li 3ndaks')
t.tag_layer(['words'])
VabamorfTagger(premorf_layer=None, postmorph_rewriter=None).tag(t)

t.morf_analysis

SL[SL[Span(tu-li, {'clitic': '', 'root_tokens': ['tu', 'li'], 'root': 'tu-li', 'ending': '0', 'partofspeech': 'Y', 'form': '?', 'lemma': 'tu-li'})],
SL[Span(3ndaks, {'clitic': '', 'root_tokens': ['3s'], 'root': '3=s', 'ending': 'ks', 'partofspeech': 'O', 'form': 'sg tr', 'lemma': '3s'})]]